In [2]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.openai import OpenAIEmbedder
from agno.tools.reasoning import ReasoningTools
from agno.knowledge import AgentKnowledge
from agno.playground import Playground, serve_playground_app

#------------------ settings ------------------
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'
embedding_model_id = 'text-embedding-v4'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

deepseek_settings = {
  'api_key' : os.getenv("DEEPSEEK_API_KEY"),
  'base_url' : os.getenv("DEEPSEEK_API_BASE_URL"),
  'id' : 'deepseek-chat'
}

settings = local_settings
#------------------ settings ------------------

In [147]:
contact_path = r"C:\Lee\work\contract\精简\锅炉合同\0111 余热锅炉商务合同.docx"
contact_path = r"C:\Lee\work\contract\精简\test\02 冷却塔订货合同.docx"
contact_path = r"C:\Lee\work\contract\精简\test\02 余热锅炉订货合同.docx"

In [148]:
import mammoth
style_map = """
p =>
b =>
i =>
"""

contact_info = mammoth.convert_to_html(contact_path,style_map=style_map,include_default_style_map=False)
contact_content = contact_info.value
print(contact_content[:500])

买方合同编号：BGZJYR-E-2019-02卖方合同编号：订 货 合 同项目名称：湛江钢铁三高炉系统工程烧结环冷余热发电系统子项名称： 余热锅炉设备供货买    方：湖南中冶长天节能环保技术有限公司卖    方： 江苏东九重工股份有限公司合同签订时间：2019年10月合同签订地点：湖南·长沙廉洁协议买方：湖南中冶长天节能环保技术有限公司卖方：江苏东九重工股份有限公司                                                                                            根据党和国家有关法律法规的各项规定，为保持廉洁自律，倡导廉洁从业，有效遏制商业贿赂行为，防止在合同签订、履行中不廉洁行为的发生，经双方协商，订立本协议，相互约束。第一条 买卖双方应当自觉遵守党和国家有关法律法规和有关廉洁自律的各项规定。第二条 买方工作人员应当保持与卖方的正常工作和业务往来，不得接受卖方的礼金、有价证券和物品，不得在卖方报销任何应由私人支付的费用。第三条 买方工作人员不得参加卖方的宴请和娱乐活动，不得违反廉洁从业的有关规定，不


In [159]:
instructions = ["严格按照expected_output中的键解析合同中的信息",
                "直接输出合同信息文本，严禁使用json格式或者```markdown```包裹",
                "严禁遗漏在expected_output中提到的任何信息",
                "合同类型：通常为 订货合同|供货合同，也可据实填写，无填None",
                "如果存在合同双方信息：分为买方和卖方单独列出",
                "如果存在最终供货设备表，禁止遗漏任何设备和价格信息",
                "如果存在分项报价表中，禁止遗漏任何设备和价格信息",
                "最终供货一览表和分项报价表按原始格式输出，(例如原始格式可能是html格式，需要保留对应的结构)"
                "最后按照expected_output中的键值对输出：买方合同编号:xxx，卖方合同编号：xxx...格式"
                ]
expected_output = '''买方合同编号
合同类型
项目名称
子项名称
买方名称
卖方名称
合同双方
  名称
  地址 
  联系人
  电话
  银行账号
  纳税人登记号
  单位名称
合同金额
不含税总价
税额
合同签订日期
最终供货一览表
分项报价表
'''
agent = Agent(model=OpenAILike(**settings,temperature=0),description="你是一位合同解析专员，严格严谨遵守约定",instructions=instructions,expected_output=expected_output,telemetry=False)

In [160]:
response = agent.run(message=contact_content)

In [ ]:
contact_exact_result = response.content.strip()
print(contact_exact_result)

买方合同编号:BGZJYR-E-2019-02
合同类型:订货合同
项目名称:湛江钢铁三高炉系统工程烧结环冷余热发电系统
子项名称:余热锅炉设备供货
买方名称:湖南中冶长天节能环保技术有限公司
卖方名称:江苏东九重工股份有限公司
合同双方
名称:湖南中冶长天节能环保技术有限公司
地址:长沙市岳麓区节庆路7号
联系人:黄淦
电话:0731—82760794
银行账号:6605 0155 2000 0246 8
纳税人登记号:9143 0100 5849 3952 00
单位名称:湖南中冶长天节能环保技术有限公司
名称:江苏东九重工股份有限公司
地址:盐城高新区华锐南路9号
联系人:刘忠
电话:0515-88597534
银行账号:5456 5820 7770
纳税人登记号:91320900140334109F
单位名称:江苏东九重工股份有限公司
合同金额:10450000.00
不含税总价:None
税额:None
合同签订日期:2019年10月15日
最终供货一览表
<table><thead><tr><th>序号</th><th>货物名称</th><th>规格型号</th><th>单位</th><th>数量</th><th>单重（吨）</th><th>总重（吨）</th><th>单   价（万元RMB）</th><th>总   价（万元RMB）</th><th>交 货 期</th><th>交货地点</th><th>交货方式</th></tr></thead><tbody><tr><td>1</td><td>余热锅炉</td><td>Q360(360)(360)/410(360)(310)-80.3(20.2)-1.8(0.5)/380(180)</td><td>套</td><td>1</td><td>912</td><td>912</td><td>1045</td><td>1045</td><td>收到预付款5个月发货，不影响安装进度前提下一个月内发完。</td><td>湛江项目现场</td><td>汽运</td></tr><tr><td colspan="12">总计：大写人民币壹仟零肆拾伍万圆整（￥10450000.00）。</td></tr></tbody></table>
分项报价表:None


In [162]:
print(len(contact_exact_result))

968


In [153]:
print(response.messages[-1].content)



买方合同编号:BGZJYR-E-2019-02
卖方合同编号:订货合同
合同类型:订货合同
项目名称:湛江钢铁三高炉系统工程烧结环冷余热发电系统
子项名称:余热锅炉设备供货
买方名称:湖南中冶长天节能环保技术有限公司
卖方名称:江苏东九重工股份有限公司
合同双方:
名称:湖南中冶长天节能环保技术有限公司
地址:长沙市岳麓区节庆路7号
联系人:黄淦
电话:0731—82760794
银行账号:6605 0155 2000 0246 8
纳税人登记号:9143 0100 5849 3952 00
单位名称:湖南中冶长天节能环保技术有限公司
名称:江苏东九重工股份有限公司
地址:盐城高新区华锐南路9号
联系人:刘忠
电话:0515-88597534
银行账号:5456 5820 7770
纳税人登记号:91320900140334109F
单位名称:江苏东九重工股份有限公司
合同金额:10450000.00
不含税总价:None
税额:None
合同签订日期:2019年10月15日
最终供货一览表(设备信息表html格式):
<table><thead><tr><th>序号</th><th>货物名称</th><th>规格型号</th><th>单位</th><th>数量</th><th>单重（吨）</th><th>总重（吨）</th><th>单价（万元RMB）</th><th>总价（万元RMB）</th><th>交货期</th><th>交货地点</th><th>交货方式</th></tr></thead><tbody><tr><td>1</td><td>余热锅炉</td><td>Q360(360)(360)/410(360)(310)-80.3(20.2)-1.8(0.5)/380(180)</td><td>套</td><td>1</td><td>912</td><td>912</td><td>1045</td><td>1045</td><td>收到预付款5个月发货，不影响安装进度前提下一个月内发完。</td><td>湛江项目现场</td><td>汽运</td></tr><tr><td colspan="12">总计：大写人民币壹仟零肆拾伍万圆整（￥10450000.00）。</td></tr></tbody></table>
分项报价表:None


In [22]:
print(response.messages[-1].reasoning_content)


好的，我现在需要处理用户提供的合同文本，并按照他们的要求提取特定信息。首先，我要仔细阅读用户提供的合同内容，确保不遗漏任何关键信息。用户要求的信息包括买方合同编号、卖方合同编号、合同类型、项目名称、子项名称、买方名称、卖方名称、合同双方信息、合同金额、不含税总价、税额、合同签订日期、最终供货一览表、分项报价表等。

首先，我需要确认买方合同编号和卖方合同编号。在文本中，买方合同编号明确给出为ZJYR3-E-2023-03，而卖方合同编号部分看起来没有提供，所以卖方合同编号应为None。接下来，合同类型是订货合同还是供货合同？根据文本开头部分提到的“订 货 合 同”，这里应该填写订货合同。

项目名称和子项名称在文本中有明确说明：项目名称是“湛江钢铁能环部3#烧结余热发电机组成套设备供货项目”，子项名称是“循环水泵”。

买方和卖方名称分别为湖南中冶长天节能环保技术有限公司和江苏诚鑫隆装备技术有限公司。合同双方的信息需要详细列出双方的地址、联系人、电话、银行账号、纳税人登记号和单位名称。这部分信息在表格中有详细内容，需要正确提取。

合同金额、不含税总价和税额在合同文本中有明确的数字，需要准确无误地复制过来。合同签订日期是2023年7月，地点是湖南·长沙，但用户要求的是合同签订日期，所以只需要年份和月份。

最终供货一览表需要转换为HTML格式的表格，这部分需要从原文中找到对应的表格，并确保结构正确。分项报价表同样在原文中存在，但可能需要检查是否有内容，比如第1、2、3年备件清单中的内容是否为空，如果为空则可能需要注明。

在处理过程中，需要注意用户强调的严格遵循expected_output，确保所有要求的字段都正确无误地呈现，使用Markdown格式，避免任何JSON格式。同时，要检查是否有遗漏或错误的信息，例如合同双方的信息是否完整，特别是卖方的信息是否在表格中正确列出。

最后，确保所有数据准确无误后，整理成用户指定的Markdown格式输出，不使用任何Markdown以外的格式，如HTML标签等。需要特别注意表格的转换是否正确，尤其是合并单元格和跨列的情况，确保最终的表格在Markdown中正确显示。



## TODO: 使用基本描述提取元数据

In [ ]:
meta_instructions = ["从合同数据中提取关键信息","主要设备名称从最终供货一览表中提取"]
meta_agent = Agent(model=OpenAILike(**local_settings,temperature=0),instructions=meta_instructions,telemetry=False)

In [ ]:
meta_agent_response = meta_agent.run(message=contact_exact_result)

In [ ]:
meta_agent_contact_exact_result = meta_agent_response
print(meta_agent_contact_exact_result)

## 结构化提取信息

本地部署的qwen输出有问题

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class ContractMeta(BaseModel):
    contact_no: str = Field(..., description="买方合同编号")
    # supplier_contact_no: str = Field(..., description="卖方合同编号")
    # contact_type: str = Field(..., description="合同类型")
    project_name: str = Field(..., description="项目名称")
    subitem_name: str = Field(..., description="子项名称")
    total_price_str: str = Field(..., description="合同金文本")
    total_price: float = Field(..., description="合同金额")
    date: str = Field(..., description="合同签订日期")
    # buyer: str = Field(..., description="买方名称")
    supplier: str = Field(..., description="卖方名称")
    main_equipments: List[str] = Field(..., description="主要设备")
    sub_equipments: List[str] = Field(..., description="分项设备")

In [164]:
meta_instructions = ["从合同数据中提取关键信息","主要设备从最终供货一览表中提取","分项设备从分项报价表中提取","数据提取严格对应，不要遗漏，不要错对提取源"]
meta_agent = Agent(model=OpenAILike(**deepseek_settings,temperature=0),instructions=meta_instructions,response_model=ContractMeta,use_json_mode=True,telemetry=False)

In [165]:
meta_agent_response = meta_agent.run(message=contact_exact_result)

In [166]:
meta_agent_contact_exact_result = meta_agent_response
print(meta_agent_contact_exact_result.content)

contact_no='BGZJYR-E-2019-02' project_name='湛江钢铁三高炉系统工程烧结环冷余热发电系统' subitem_name='余热锅炉设备供货' supplier='江苏东九重工股份有限公司' main_equipments=['余热锅炉'] sub_equipments=[]


In [167]:
if isinstance(meta_agent_contact_exact_result.content,ContractMeta):
    contract_meta:ContractMeta = meta_agent_contact_exact_result.content
    contact_meta_str = contract_meta.model_dump()
    print(contact_meta_str)


{'contact_no': 'BGZJYR-E-2019-02', 'project_name': '湛江钢铁三高炉系统工程烧结环冷余热发电系统', 'subitem_name': '余热锅炉设备供货', 'supplier': '江苏东九重工股份有限公司', 'main_equipments': ['余热锅炉'], 'sub_equipments': []}


In [ ]:
# 键名映射表
mapping = {
    'contact_no': '合同编号',
    'project_name': '项目名称',
    'subitem_name':'子项名称',
    'supplier': '供应商',
    'main_equipments':'主要设备',
    'sub_equipments':'分项设备'
}

list_chunk_size = 10 # 设备列表截断数，防止语义向量化之后被平均

# 转换函数
def dict_to_str_with_mapping(d, mapping):
    items = []
    for k, v in d.items():
        new_key = mapping.get(k, k)
        if isinstance(v, list):
            v_str = ','.join(str(i) for i in v[:list_chunk_size])
        else:
            v_str = str(v)
        items.append(f"{new_key}: {v_str}")
    return ' '.join(items)

# 使用示例
result = dict_to_str_with_mapping(contact_meta_str, mapping)
print(result)
print(len(result))

合同编号: BGZJYR-E-2019-02 项目名称: 湛江钢铁三高炉系统工程烧结环冷余热发电系统 子项名称: 余热锅炉设备供货 供应商: 江苏东九重工股份有限公司 主要设备: 余热锅炉 分项设备: 
101
